In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pyexplain.benchmark.file_utils import *
from pyexplain.benchmark.plot import *
from pyexplain.benchmark.check_results import *
import IPython
import numpy as np
import matplotlib.pyplot as plt
from pyexplain.examples.utils import *
import pandas as pd
from collections import Counter, defaultdict
pd.set_option('display.max_rows', 100)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.weight']= 'bold'



from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
display(HTML("<style>.output_result { max-width:85% !important; }</style>"))

# MAC
environment = 'MAC'
BASE_MAC_LINUX = {
    'MAC': '/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects/',
    'LINUX': '/home/emilio/research/'
}

PATH_FIGURES_POST_PAPER = Path(BASE_MAC_LINUX[environment]) / "01_holygrail/latex/journal/jair21/figures/"
EXPERIMENT_RESULTS = Path(BASE_MAC_LINUX[environment]) / "06_HPC_Experiments/experiments/data/output/"
BASE_OUTPUT_PATH = BASE_MAC_LINUX[environment] + "06_HPC_Experiments/experiments/data/output/"
REMOTE_EXPERIMENT_RESULTS = "/data/brussel/101/vsc10143/hpc_experiments2/experiments/data/output/"
OCUS_EXPLAIN_EXAMPLES = Path(BASE_MAC_LINUX[environment] + "05_OCUS_Explain/code/pyexplain/examples/")


In [2]:
DISABLE_KILLER_INSTANCES = False

In [ ]:
df_all_grows = pd.read_pickle("/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects/06_HPC_Experiments/jair/jair_rq2_all_grows.pkl")

In [ ]:
df_mus = pd.read_pickle("/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects/06_HPC_Experiments/jair/jair_rq2_mus.pkl")

# RQ2 Which domain specific grow improve the overall runtime?

using the cumulative runtime computed directly for the new dataset

In [ ]:
df_all_grows[
    (df_all_grows["params_instance"].str.contains("sudoku-easy")) &
    (df_all_grows["params_grow_config"] == "SUBSETMAX SAT MCSes + Actual + Unif.") &
    (df_all_grows["explanation config"] == "OUS Iter.+Lit. Incr. HS")
][["params_instance", "time_totalTime", "time_to_first_expl"]]

# Incremental vs non-incremental runtime plot

In [ ]:
num_instances = len(set(df_all_grows["params_instance"]))

for id, row in df_all_grows[
                    ["explanation config", "params_grow_config", "time_totalTime"]
                ].groupby(by=["explanation config", "params_grow_config"]).count().iterrows():
    assert row["time_totalTime"] == num_instances, f'Num instances for config {row["explanation config"]} - {row["params_grow_config"]}: expected = {num_instances} got {row["time_totalTime"]}'


In [ ]:
sat_grow = 'SAT'
# grow = 'SAT MCSes + Actual + Unif.'
# fixing the grow: sat correction subset enumeration
df_sat_corr_grow = df_all_grows[
    df_all_grows["params_grow_config"] == sat_grow
]
# display(df_all_grows["time_totalTime"])

mapping_expl_config_incremental = {
    'OCUS': 'OCUS+Incr. HS',
    'OUS Iter.': 'OUS Iter.+Lit. Incr. HS',
    'OUSb': 'OUSb+Lit. Incr. HS'
}

label_mapping_expl_config = {
    'OCUS': 'OCUS',
    'OUS Iter.': 'OCUS\_Split',
    'OUSb': 'OCUS\_Bounded'
}

color_matching = {
    'OCUS': 'cornflowerblue',
    'OUS Iter.': 'tab:orange',
    'OUS Iter.+Lit. Incr. HS': 'tab:red',
    'OUSb': 'forestgreen',
    'MUS': 'tab:brown',
    'OUSb+Lit. Incr. HS': 'forestgreen',
    'OCUS+Incr. HS': 'tab:blue'
}


all_instances = set(df_sat_corr_grow["params_instance"])

plt.figure(figsize=(6,6))

all_worse_instances = set()

for non_incremental_config, incremental_config in mapping_expl_config_incremental.items():
    non_incremental_times = []
    incremental_times = []
    for instance in all_instances:
        incremental_runtime = df_sat_corr_grow[
            (df_sat_corr_grow["params_instance"] == instance) &
            (df_sat_corr_grow["explanation config"] == incremental_config)
        ]["time_totalTime"]
                
        non_incremental_runtime = df_sat_corr_grow[
            (df_sat_corr_grow["params_instance"] == instance) &
            (df_sat_corr_grow["explanation config"] == non_incremental_config)
        ]["time_totalTime"]

        incremental_times.append(float(incremental_runtime))
        non_incremental_times.append(float(non_incremental_runtime))
        
        assert len(incremental_runtime) == 1, "Incremnetal time empty"
        assert len(non_incremental_runtime) == 1, "Non-Incremnetal time empty"
        
        if float(incremental_runtime) > float(non_incremental_runtime) and incremental_config == "OUS Iter.+Lit. Incr. HS":
            print(f"{incremental_config=} \t{instance} - {float(incremental_runtime)} s") 
            print(f"{non_incremental_config=} \t{instance} - {float(non_incremental_runtime)} s") 
            all_worse_instances.add(instance)
        
    plt.scatter(
        incremental_times, 
        non_incremental_times, 
        marker='+',
        label=label_mapping_expl_config[non_incremental_config],
        color=color_matching[incremental_config]
    )

plt.plot(range(1, 4000, 100), range(1, 4000, 100), '-', color='black')
plt.xlim([1, 4000]);plt.ylim([1, 4000])
plt.xscale('log');plt.yscale('log')
plt.legend(loc="lower right", fontsize=20)
plt.xlabel('Incremental OCUS - time (s)', fontsize=22); plt.ylabel('Non-incremental OCUS - time (s)', fontsize=20)
plt.xticks(fontsize=20); plt.yticks(fontsize=22)


filepath = PATH_FIGURES_POST_PAPER / "incremental" / datetime.now().strftime(f"incremental_vs_non_{sat_grow}_%Y_%m_%d.pdf")
print(filepath)
plt.savefig(
    PATH_FIGURES_POST_PAPER / "incremental" / datetime.now().strftime(f"incremental_vs_non_{sat_grow}_%Y_%m_%d.pdf"),bbox_inches ="tight",
            transparent = True
)

# Correction Subset Enumeraiton strategies

In [ ]:
ignored_grows = [
 'SUBSETMAX SAT MCSes + Final + Unif.',
 'SUBSETMAX SAT MCSes + Full + Unif.',
 'SUBSETMAX SAT MCSes + Initial + Unif.',
    'Greedy MCSes + Actual + Unif.'
    
# 'Greedy-Sat + Final + Unif.',
 #'Greedy-Sat + Full + Unif.',
 #'Greedy-Sat + Initial + Unif.',
]

grow_renaming =  {
    'MaxSAT + Actual + Unif.': 'Single Dom.-spec. MaxSAT',
    'MaxSAT + Full + Unif.': 'Single MaxSAT Full',

    'SAT': 'Single SAT',

    'Greedy MCSes + Actual + Unif.': 'Dom.-spec. MaxSAT+Multi SAT',

    'Disj.MCSes + Actual + Unif.': 'Multi Dom.-spec.MaxSAT',

    'SAT MCSes + Actual + Unif.': "Multi SAT",

    'SUBSETMAX SAT MCSes + Actual + Unif.': "Multi SubsetMax-SAT",
    'SUBSETMAX SAT MCSes + Final + Unif.': "Multi SubsetMax-SAT.-Final",
    'SUBSETMAX SAT MCSes + Full + Unif.': "Multi SubsetMax SAT.-Full",
    'SUBSETMAX SAT MCSes + Initial + Unif.': "Multi SubsetMax-SAT.-Initial",

    'Greedy-Sat + Actual + Unif.': 'Single SubsetMax-SAT',
    'Greedy-Sat + Final + Unif.': 'Single SubsetMax-SAT-Final',
    'Greedy-Sat + Full + Unif.': 'Single SubsetMax-SAT-Full',
    'Greedy-Sat + Initial + Unif.': 'Single SubsetMaxSAT-Initial',
    
    "MUS": 'MUS'
}

## PUT MUS ALWAYS AS BOTTOM ON LEGEND
grow_sorted_ordering = [
    'OUSb',
    'OUSb+Lit. Incr. HS',
    'OCUS',
    'OCUS+Incr. HS',
    'OUS Iter.',
    'OUS Iter.+Lit. Incr. HS',
    'MUS',
]

color_matching = {
    'OCUS': 'cornflowerblue',
    'OUS Iter.': 'tab:orange',
    'OUS Iter.+Lit. Incr. HS': 'tab:red',
    'OUSb': 'forestgreen',
    'MUS': 'tab:brown',
    'OUSb+Lit. Incr. HS': 'forestgreen',
    'OCUS+Incr. HS': 'tab:blue'
}

epxl_conf_renaming = {
    'OCUS': 'OCUS',
    'OUS Iter.': 'OCUS\_Split',
    'OUS Iter.+Lit. Incr. HS': 'OCUS\_Split+Incr.',
    'OUSb': 'OCUS\_Bounded',
    'MUS': 'MUS',
    'OUSb+Lit. Incr. HS': 'OCUS\_Bounded+Incr.',
    'OCUS+Incr. HS': 'OCUS+Incr.'
}


conf_title = {
    "OUS Iter.+Lit. Incr. HS": "Expl.-Spec. OCUS+Incr.",
    "OCUS": "OCUS",
    "OUS Iter.": "Expl.-Spec. OCUS",
    "OCUS+Incr. HS": "OCUS+Incr.",
    "OUSb": "Bounded OUS",
    "OUSb+Lit. Incr. HS": "Bounded OUS+Incr.",
}

In [ ]:


color_matching = {
    'OCUS': 'tab:blue',
    'OUS Iter.': 'tab:red',
    'OUS Iter.+Lit. Incr. HS': 'tab:red',
    'OUSb': 'forestgreen',
    'MUS': 'tab:brown',
    'OUSb+Lit. Incr. HS': 'forestgreen',
    'OCUS+Incr. HS': 'tab:blue'
}

### MUS Time to explain instances
mus_time_instances = sorted(list(df_all_grows[
    (df_all_grows["time_timedout"] == 0) & 
    (df_all_grows["explanation config"] == "MUS")
]["time_totalTime"]))

## COMPUTING ORDERING FOR ALL CONFIGS - "Greedy MCSes + Actual + Unif." as best grow
grow_config_timings = []

for conf in set(df_all_grows["explanation config"]):
    if conf == "MUS": continue
    time_instances = sorted(list(df_all_grows[
            (df_all_grows["time_timedout"] == 0) & 
            (df_all_grows["params_grow_config"] == "Greedy MCSes + Actual + Unif.") &
            (df_all_grows["explanation config"] == conf)
        ]["time_totalTime"]))
    
    grow_config_timings.append((conf, len(time_instances), max([10**8] + time_instances), time_instances))



for grow in set(df_all_grows["params_grow_config"]) - set(ignored_grows):
        
    grow_timings = {
        "MUS": (
            len(mus_time_instances), 
            max([1000000000] + mus_time_instances), 
            mus_time_instances
        )
    }
    
    
    ## select 
    df_selected_grow = df_all_grows[df_all_grows["params_grow_config"] == grow]
    for conf in set(df_selected_grow["explanation config"]):
        if conf == "MUS": continue
        
        time_instances = sorted(list(df_selected_grow[
            (df_selected_grow["time_timedout"] == 0) & 
            (df_selected_grow["explanation config"] == conf)
        ]["time_totalTime"]))
        
        if len(time_instances) == 0: continue
        
        grow_timings[conf] = (len(time_instances), max([1000000000] + time_instances), time_instances)

    
    ### BUILDING FIGURE

    
    plt.figure(figsize=(9, 6))
    for conf in grow_sorted_ordering:
        if conf not in grow_timings: continue

        _,_, time_instances = grow_timings[conf]
        if 'Incr' in conf:
            plt.plot(
                range(1, len(time_instances)+1), 
                time_instances, 
                label=epxl_conf_renaming[conf], 
                linewidth=2.5, 
                color=color_matching[conf]
            )
        else:
            plt.plot(
                range(1, len(time_instances)+1), 
                time_instances, 
                '-.', 
                label=epxl_conf_renaming[conf], 
                linewidth=2.25, 
                color=color_matching[conf]
            )
    
    plt.legend(fontsize=18, loc='center left', bbox_to_anchor=(1.0, 0.55))
    #plt.title(grow_renaming[grow])
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.ylim([1, 4000])
    plt.yscale('log')
    
    plt.ylabel("Time (s)",fontsize=24)
    plt.xlabel("Number of instances solved",fontsize=24)
    output_filename = str(PATH_FIGURES_POST_PAPER / "grow" / f"{datetime.now().strftime(f'rq3_cactus_r_%Y_%m_%d')}_{grow.replace(' ', '_').replace('+', '')}.pdf")
    print(output_filename)
    plt.savefig(output_filename, bbox_inches='tight')

# Effect of the grow method on incrementality!

In [ ]:
df_all_grows[
    (df_all_grows["time_timedout"]) &
    (df_all_grows["params_instance"].str.contains("sudoku")) &
    (~df_all_grows["params_instance"].str.contains("wiki")) &
    (~df_all_grows["params_instance"].str.contains("killer"))
    #df_all_grows["params_explanation_computer"].str.contains("OCUS")
][
    ["params_instance", "tot_lits_derived", "time_timedout"]
].sort_values(
    by=["params_instance", "tot_lits_derived"], 
    ascending=[True, False]
)

In [ ]:
print(set(df_all_grows["params_grow_config"]))

In [ ]:
ignored_grow_configurations = [
    'Disj.MCSes + Full + Unif.',
    # 'Greedy MCSes + Full + Unif.' ,
    'No Grow',
    'SUBSETMAX SAT MCSes + Final + Unif.',
    'SUBSETMAX SAT MCSes + Full + Unif.',
    'SUBSETMAX SAT MCSes + Initial + Unif.',
    'Greedy-Sat + Final + Unif.',
    'Greedy-Sat + Full + Unif.',
    'Greedy-Sat + Initial + Unif.',
    'Greedy MCSes + Actual + Unif.'
]


selected_grow_configurations = [
    c for c in set(df_all_grows["params_grow_config"]) if c not in ignored_grow_configurations
]

select_explanation_configs = list(set(df_all_grows["explanation config"]))

mus_time_instances = time_instances = sorted(list(df_all_grows[
    (~df_all_grows["time_timedout"]) & 
    (df_all_grows["explanation config"] == "MUS")
]["time_totalTime"]))

## COMPUTING ORDERING FOR ALL CONFIGS - "Greedy MCSes + Actual + Unif." as best grow
grow_config_timings = []

for grow in selected_grow_configurations:
    if grow == "MUS": continue
    time_instances = sorted(list(df_all_grows[
            (~df_all_grows["time_timedout"]) & 
            (df_all_grows["params_grow_config"] == grow) &
            (df_all_grows["explanation config"] == "OUS Iter.+Lit. Incr. HS")
        ]["time_totalTime"]))
    
    grow_config_timings.append((grow, len(time_instances), max([10**8] + time_instances), time_instances))

## PUT MUS ALWAYS AS BOTTOM ON LEGEND
grow_sorted_ordering = [
	"SAT",
    "MaxSAT + Actual + Unif.",
	"MaxSAT + Full + Unif.",
    'Greedy-Sat + Final + Unif.',
    'Greedy-Sat + Full + Unif.',
    'Greedy-Sat + Initial + Unif.',
	"Greedy-Sat + Actual + Unif.",
    "Disj.MCSes + Actual + Unif.",
	"Greedy MCSes + Actual + Unif.",
	"SUBSETMAX SAT MCSes + Actual + Unif.",
	"SUBSETMAX SAT MCSes + Full + Unif.",
	"SUBSETMAX SAT MCSes + Initial + Unif.",
	"SUBSETMAX SAT MCSes + Final + Unif.",
    'SAT MCSes + Actual + Unif.',
    "MUS",
]

for conf in select_explanation_configs:
    if conf == "MUS": continue

    plt.figure(figsize=(8,6))
    
    explanation_timing = {
        "MUS": (len(mus_time_instances), max([1000000000] + mus_time_instances), mus_time_instances)
    }
    print("selected_grow_configurations:")
    for grow in selected_grow_configurations:
        print("\t", conf, grow)
        #
        if grow == "MUS": continue

        time_instances = sorted(list(df_all_grows[
            (~df_all_grows["time_timedout"]) & 
            (df_all_grows["params_grow_config"] == grow) &
            (df_all_grows["explanation config"] == conf)
        ]["time_totalTime"]))
        print("\t", grow, len(time_instances))
        
        if len(time_instances) == 0: continue
        explanation_timing[grow] = (len(time_instances), max([1000000000] + time_instances), time_instances)

    base_grows = ['Dom.-spec. MaxSAT', 'MaxSAT Full', 'No grow']
    print(list(explanation_timing))
    print("grow_sorted_ordering:")
    for grow in grow_sorted_ordering:
        print("\t", conf, grow)
        if grow not in explanation_timing: continue
        
        _,_, time_instances = explanation_timing[grow]
        renamed_grow = grow_renaming[grow] if (grow in grow_renaming) else grow
        if renamed_grow == "MUS" or renamed_grow in base_grows:
            plt.plot(range(1, len(time_instances)+1), time_instances, '-.', label=renamed_grow, linewidth=2)
        else:
            plt.plot(range(1, len(time_instances)+1), time_instances, label=renamed_grow, linewidth=2)
#     plt.legend(fontsize=18, loc='lower right')#
    if conf == "OCUS+Incr. HS":
        plt.legend(fontsize=20, loc='upper left', bbox_to_anchor=(1.2, 0.95))

#     def export_legend(legend, filename=str(PATH_FIGURES_POST_PAPER /"legend.png")):
#         fig  = legend.figure
#         fig.canvas.draw()
#         bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
#         fig.savefig(filename, dpi="figure", bbox_inches=bbox)

#     export_legend(
#         legend,
#         filename=str(PATH_FIGURES_POST_PAPER /f"legend_{conf.replace(' ', '_').replace('+', '').replace('.','')}.pdf"))
    #TODO: add description outside of plot
#     plt.title(f'{conf_title[conf]}',fontsize=22)
#     print(conf)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.yscale('log')
    plt.ylabel("Time (s)",fontsize=24)
    plt.ylim([1, 10000])
    plt.xlabel("Number of instances solved",fontsize=24)
    output_filename = str(PATH_FIGURES_POST_PAPER / "effect_grow_incremental"/ f"{datetime.now().strftime(f'rq2_cactus_log_%Y_%m_%d')}_{conf.replace(' ', '_').replace('+', '').replace('.','')}.pdf")
    print(output_filename)
    plt.savefig(output_filename, bbox_inches='tight')

## For the best configuration - Corr Subsets - generate cumulative explanation time

## SUBSETMAX SAT MCSes and MUS

In [ ]:
# df_all_grows
df_mus_all_grows_selected = df_all_grows[
    (df_all_grows["params_grow_config"].isin(["SUBSETMAX SAT MCSes + Actual + Unif.", "MUS"])) & 
    (~df_all_grows["time_timedout"])
]

## df_corr_selected
# display(df_mus_all_grows_selected.groupby(by=["params_explanation_computer","params_grow_config"]).count())

df_corr_selected_grouped = df_mus_all_grows_selected.groupby(by=["params_explanation_computer", "params_grow_config"]).agg(
    t=("tot_time_explain" , lambda x: round(mean_ignore_zeros(x), 2)),
    f=("time_timedout", lambda x: f"[{str(len(x) - np.sum(x)).zfill(3)} / {len(x)}]"),
    avg_opt=("%time_opt2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_opt=("%time_opt2" , lambda x: round(np.std(x), 2)),
    avg_sat=("%time_sat2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_sat=("%time_sat2" ,lambda x: round(np.std(x), 2)),
    avg_grow=("%time_grow2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_grow=("%time_grow2" , lambda x: round(np.std(x), 2)),
    avg_disj_mcs=("%time_disj_mcs2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_disj_mcs=("%time_disj_mcs2" , lambda x: round(np.std(x), 2)),
    avg_remaining=("%time_remaining2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_remaining=("%time_remaining2" , lambda x: round(np.std(x), 2)),
    sum_t_opt=("tot_time_opt" , lambda x: sum(x)),
    sum_t_sat=("tot_time_sat" , lambda x: sum(x)),
    sum_t_grow=("tot_time_grow" , lambda x: sum(x)),
    avg_n_opt=("tot_n_opt" , lambda x: round(mean_ignore_zeros(x))),
    avg_n_sat=("tot_n_sat" , lambda x: round(mean_ignore_zeros(x))),
    avg_n_grow=("tot_n_grow" , lambda x: round(mean_ignore_zeros(x))),
    tot_n_opt=("tot_n_opt" , lambda x: sum(x)),
    tot_n_sat=("tot_n_sat" , lambda x: sum(x)),
    tot_n_grow=("tot_n_grow" , lambda x: sum(x)),
    avg_n_hs=("tot_n_hs" , lambda x: round(mean_ignore_zeros(x))),
    cnt = ("time_timedout", "count")
)

df_temp = df_corr_selected_grouped.reset_index()
for k in ["opt", "sat", "grow", "disj_mcs", "remaining"]:
    df_temp[k] = df_temp.apply(lambda row: None, axis=1)
    df_temp["%"+k] = df_temp.apply(lambda row: None, axis=1)

df_temp["avg_t_opt"] = df_temp.apply(
    lambda row: round(row["sum_t_opt"]/row['tot_n_opt'], 4) if row["tot_n_opt"] !=0 else 0, axis=1)
df_temp["avg_t_sat"] = df_temp.apply(
    lambda row: round(row['sum_t_sat']/row['tot_n_sat']  if row["tot_n_sat"] !=0 else 0, 4), axis=1)
df_temp["avg_t_grow"] = df_temp.apply(
    lambda row: round(row['sum_t_grow']/row['tot_n_grow']  if row["tot_n_grow"] !=0 else 0, 4), axis=1)

for index,row in df_temp.iterrows():
    for k in ["opt", "sat", "grow", "disj_mcs", "remaining"]:
        df_temp.at[index,k]= f'{row["avg_"+k]}% [+/- {row["std_"+k]}%]'
        df_temp.at[index,"%"+k]= f'{row["avg_"+k]}%'

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}
        
df_temp["expl_config"] = df_temp.apply(
    lambda row: renaming_expl_config[row['params_explanation_computer']], axis=1)

percentage_exec_time_ous = df_temp[
        df_temp["expl_config"] != "bla"
    ][
    ["expl_config",#'params_grow_config',
     "t",
     "f",
     "%opt", 
     "%sat", 
     "%grow", 
     "%disj_mcs",
     #"%remaining",
     "avg_n_hs"
     #'avg_t_opt', 
     #'avg_n_opt', 
     #'avg_t_sat',
     #'avg_n_sat', 
     #'avg_t_grow',
     #'avg_n_grow'
    ]].sort_values(["avg_n_hs"], ascending=[False])
display(percentage_exec_time_ous)
print(percentage_exec_time_ous.to_latex(index=False))

# df_sat_ignored_instances.groupby(by=["params_explanation_computer"]).sum()

## Subset max SAT grow and MUS

In [ ]:
# df_all_grows
df_mus_all_grows_selected = df_all_grows[
    (df_all_grows["params_grow_config"].isin(["Greedy-Sat + Actual + Unif.", "MUS"])) & 
    (~df_all_grows["time_timedout"])
]

## df_corr_selected
# display(df_mus_all_grows_selected.groupby(by=["params_explanation_computer","params_grow_config"]).count())

df_corr_selected_grouped = df_mus_all_grows_selected.groupby(by=["params_explanation_computer", "params_grow_config"]).agg(
    t=("tot_time_explain" , lambda x: round(mean_ignore_zeros(x), 2)),
    f=("time_timedout", lambda x: f"[{str(len(x) - np.sum(x)).zfill(3)} / {len(x)}]"),
    avg_opt=("%time_opt2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_opt=("%time_opt2" , lambda x: round(np.std(x), 2)),
    avg_sat=("%time_sat2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_sat=("%time_sat2" ,lambda x: round(np.std(x), 2)),
    avg_grow=("%time_grow2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_grow=("%time_grow2" , lambda x: round(np.std(x), 2)),
    avg_disj_mcs=("%time_disj_mcs2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_disj_mcs=("%time_disj_mcs2" , lambda x: round(np.std(x), 2)),
    avg_remaining=("%time_remaining2" , lambda x: round(mean_ignore_zeros(x), 2)),
    std_remaining=("%time_remaining2" , lambda x: round(np.std(x), 2)),
    sum_t_opt=("tot_time_opt" , lambda x: sum(x)),
    sum_t_sat=("tot_time_sat" , lambda x: sum(x)),
    sum_t_grow=("tot_time_grow" , lambda x: sum(x)),
    avg_n_opt=("tot_n_opt" , lambda x: round(mean_ignore_zeros(x))),
    avg_n_sat=("tot_n_sat" , lambda x: round(mean_ignore_zeros(x))),
    avg_n_grow=("tot_n_grow" , lambda x: round(mean_ignore_zeros(x))),
    tot_n_opt=("tot_n_opt" , lambda x: sum(x)),
    tot_n_sat=("tot_n_sat" , lambda x: sum(x)),
    tot_n_grow=("tot_n_grow" , lambda x: sum(x)),
    avg_n_hs=("tot_n_hs" , lambda x: round(mean_ignore_zeros(x))),
    cnt = ("time_timedout", "count")
)

df_temp = df_corr_selected_grouped.reset_index()
for k in ["opt", "sat", "grow", "disj_mcs", "remaining"]:
    df_temp[k] = df_temp.apply(lambda row: None, axis=1)
    df_temp["%"+k] = df_temp.apply(lambda row: None, axis=1)

df_temp["avg_t_opt"] = df_temp.apply(
    lambda row: round(row["sum_t_opt"]/row['tot_n_opt'], 4) if row["tot_n_opt"] !=0 else 0, axis=1)
df_temp["avg_t_sat"] = df_temp.apply(
    lambda row: round(row['sum_t_sat']/row['tot_n_sat']  if row["tot_n_sat"] !=0 else 0, 4), axis=1)
df_temp["avg_t_grow"] = df_temp.apply(
    lambda row: round(row['sum_t_grow']/row['tot_n_grow']  if row["tot_n_grow"] !=0 else 0, 4), axis=1)

for index,row in df_temp.iterrows():
    for k in ["opt", "sat", "grow", "disj_mcs", "remaining"]:
        df_temp.at[index,k]= f'{row["avg_"+k]}% [+/- {row["std_"+k]}%]'
        df_temp.at[index,"%"+k]= f'{row["avg_"+k]}%'

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}
        
df_temp["expl_config"] = df_temp.apply(
    lambda row: renaming_expl_config[row['params_explanation_computer']], axis=1)

percentage_exec_time_ous = df_temp[
        df_temp["expl_config"] != "bla"
    ][
    ["expl_config",#'params_grow_config',
     "t",
     "f",
     "%opt", 
     "%sat", 
     "%grow", 
     "%disj_mcs",
     #"%remaining",
     "avg_n_hs"
     #'avg_t_opt', 
     #'avg_n_opt', 
     #'avg_t_sat',
     #'avg_n_sat', 
     #'avg_t_grow',
     #'avg_n_grow'
    ]].sort_values(["avg_n_hs"], ascending=[False])
display(percentage_exec_time_ous)
print(percentage_exec_time_ous.to_latex(index=False))

# df_sat_ignored_instances.groupby(by=["params_explanation_computer"]).sum()

## RQ4: What is the efficiency of a single step O(C)US and is single step sufficiently efficient for an interactive context?

### RQ4a) Time to first explanation and average time to explain for subsetmax ___sat correction subsets___

Ingoring the timedout instances


In [ ]:
df_greedy_corr =  df_all_grows[
    #(df_all_grows["params_grow_config"] == "SUBSETMAX SAT MCSes + Actual + Unif.")
    (df_all_grows["params_grow_config"] == "Greedy MCSes + Actual + Unif.")   
]

grouped_params_instance_summed_incremental = df_greedy_corr.groupby(
    by=["params_instance"]
).sum().reset_index()

# non_timedout_instances_incremental = set(df_corr_enh["params_instance"])
non_timedout_instances_incremental = grouped_params_instance_summed_incremental[
    grouped_params_instance_summed_incremental["time_timedout"] == 0
]["params_instance"].to_list()

df_greedy_corr_incremental_non_timeout = df_greedy_corr[
    df_greedy_corr["params_instance"].isin(non_timedout_instances_incremental)
]

grouped_greedy_corr_configs = df_greedy_corr_incremental_non_timeout.groupby(
    by=["explanation config"]
).agg(
    t_expl=("time_explain", sum),
    t_avg_first_expl=("time_to_first_expl", lambda x: round(np.mean(x), 2)),
    cnt=("time_explain", "count"),
    tot_time=("tot_time_explain", np.sum)
).reset_index()


df_incr_configs_indiv_expls = stats_expls_times(
    grouped_greedy_corr_configs[
        [
            "explanation config", 
            "t_expl", 
            "t_avg_first_expl",
            "tot_time"]
    ].copy(deep=True)
)

ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_incr_configs_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_incr_configs_indiv_expls[
    [
        "explanation config", 
        #"t_avg",
        "t_avg_first_expl",
        "avg_t_expl", 
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]

display(df_indiv_expls_table)
print(df_indiv_expls_table.to_latex(index=False))

In [ ]:
### All instances where the incremental version performs worse that the non-incremnetal

df_sat_grow_corr =  df_all_grows[
    #(df_all_grows["params_grow_config"] == "SUBSETMAX SAT MCSes + Actual + Unif.")
    (df_all_grows["params_grow_config"] == 'SAT') &
    (df_all_grows["params_instance"].isin(all_worse_instances))
]

print(df_all_grows["time_to_first_expl"])
print(df_sat_grow_corr["time_to_first_expl"])

grouped_params_instance_summed_incremental = df_sat_grow_corr.groupby(
    by=["params_instance"]
).sum().reset_index()

# non_timedout_instances_incremental = set(df_corr_enh["params_instance"])
non_timedout_instances_incremental = grouped_params_instance_summed_incremental[
    grouped_params_instance_summed_incremental["time_timedout"] == 0
]["params_instance"].to_list()

df_sat_grow_corr_incremental_non_timeout = df_sat_grow_corr[
    df_sat_grow_corr["params_instance"].isin(non_timedout_instances_incremental)
]

grouped_sat_grow_corr_configs = df_sat_grow_corr_incremental_non_timeout.groupby(
    by=["explanation config"]
).agg(
    t_expl=("time_explain", sum),
    t_avg_first_expl=("time_to_first_expl", lambda x: round(np.mean(x), 2)),
    cnt=("time_explain", "count"),
    tot_time=("tot_time_explain", np.sum)
).reset_index()


display(grouped_sat_grow_corr_configs)

df_incr_configs_indiv_expls = stats_expls_times(
    grouped_sat_grow_corr_configs[
        [
            "explanation config", 
            "t_expl", 
            "t_avg_first_expl",
            "tot_time"]
    ].copy(deep=True)
)

ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_incr_configs_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_incr_configs_indiv_expls[
    [
        "explanation config", 
        #"t_avg",
        "t_avg_first_expl",
        "avg_t_expl", 
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]

display(df_indiv_expls_table)
print(df_indiv_expls_table.to_latex(index=False))

### RQ4b) Time to first explanation and average time to explain for subsetmax ___sat correction subsets___

In [ ]:
### non-timeout instances for all configurations 
df_non_timeout_instances_subset_max_sat = df_greedy_corr =  df_all_grows[
    (df_all_grows["params_grow_config"] == "SUBSETMAX SAT MCSes + Actual + Unif.")
].groupby(["params_instance"]).sum().reset_index()

non_timeout_instances_subset_max_sat = df_non_timeout_instances_subset_max_sat[
    df_non_timeout_instances_subset_max_sat["time_timedout"] == 0
]["params_instance"].to_list()

In [ ]:

df_greedy_corr =  df_all_grows[
    (df_all_grows["params_grow_config"] == "Greedy MCSes + Actual + Unif.")&

    # (df_all_grows["params_grow_config"] == "SUBSETMAX SAT MCSes + Actual + Unif.") &
    (df_all_grows["params_instance"].isin(non_timeout_instances_subset_max_sat))
]



grouped_greedy_corr_configs = df_greedy_corr.groupby(
    by=["explanation config"]
).agg(
    t_expl=("time_explain", sum),
    t_avg_first_expl=("time_to_first_expl", lambda x: round(np.mean(x), 2)),
    cnt=("time_explain", "count"),
    tot_time=("tot_time_explain", np.sum)
).reset_index()


df_incr_configs_indiv_expls = stats_expls_times(
    grouped_greedy_corr_configs[
        [
            "explanation config", 
            "t_expl", 
            "t_avg_first_expl",
            "tot_time"]
    ].copy(deep=True)
)

ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_incr_configs_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_incr_configs_indiv_expls[
    [
        "explanation config", 
        #"t_avg",
        "t_avg_first_expl",
        "avg_t_expl", 
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]

display(df_indiv_expls_table)
print(df_indiv_expls_table.to_latex(index=False))

### Average Improvement speed

This is sad because there are some isntances where a mus approach is too expensive :(

In [25]:
t_mus, t_ocus, t_diff = [], [], []

df_mus_ocus = df_all_grows[
    (df_all_grows["explanation config"].isin(["OUS Iter.+Lit. Incr. HS", "MUS"])) & 
    (df_all_grows["params_grow_config"].isin(["SUBSETMAX SAT MCSes + Actual + Unif.", "MUS"]))
]
display(df_mus_ocus.groupby(by=["explanation config"]).count())

for instance in set(df_mus_ocus["params_instance"]):
    t_mus_instance = float(df_mus_ocus[
        (df_mus_ocus["params_instance"] == instance) &
        (df_mus_ocus["explanation config"] == "MUS")
    ]["tot_time_explain"])
    
    t_ocus_instance = float(df_mus_ocus[
        (df_mus_ocus["params_instance"] == instance) &
        (df_mus_ocus["explanation config"] == "OUS Iter.+Lit. Incr. HS")
    ]["tot_time_explain"])
    
    t_mus.append(t_mus_instance)
    t_ocus.append(t_ocus_instance)
    t_diff_instance = (t_mus_instance - t_ocus_instance)/t_mus_instance
    t_diff.append(t_diff_instance * 100)
    #print(t_mus_instance, t_ocus_instance, t_diff_instance * 100)
        
print(f"Average speed-up from MUS -> OCUS best configuration: {round(np.mean(t_diff))}%")

,time_totalTime,time_timeout,time_timedout,time_explain,time_cumul_explain,time_preprocess,time_opt,time_sat,time_grow,time_disj_mcs,...,avg_t_explain,max_t_explain,min_t_explain,n_expls,%time_remaining_ocus,corr_explanation_config,incremental,HS,time_to_first_expl,time_tavg_greedy_explain
explanation config,,,,,,,,,,,,,,,,,,,,,
MUS,403,403,403,403,403,403,0,0,0,0,...,389,403,403,403,403,403,403,0,403,403
OUS Iter.+Lit. Incr. HS,403,403,403,403,403,403,403,403,403,403,...,388,403,403,403,403,403,403,403,403,0


Average speed-up from MUS -> OCUS best configuration: 56%
